https://github.com/google-research/bert

https://arxiv.org/pdf/1810.04805.pdf

We introduce a new language representation model called BERT, which stands for
Bidirectional Encoder Representations from
Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from
unlabeled text by jointly conditioning on both
left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer
to create state-of-the-art models for a wide
range of tasks, such as question answering and
language inference, without substantial taskspecific architecture modifications.

### Comprensión de Contexto Bidireccional
A diferencia de los modelos tradicionales que leen secuencias de texto en una dirección (ya sea de izquierda a derecha o de derecha a izquierda), BERT lee la secuencia completa de palabras de una vez. Este enfoque bidireccional permite que el modelo entienda el contexto de una palabra basándose en todo su entorno (tanto a la izquierda como a la derecha de la palabra).

### Diferencias del Transformador Original
#### Uso de Solo el Codificador
El modelo Transformador original consta de un codificador y un decodificador. BERT utiliza únicamente la pila de codificadores del Transformador.

#### Bidireccionalidad
El Transformador en su forma original no es inherentemente bidireccional de la manera en que lo es BERT. Procesa el texto de manera secuencial (ya sea de izquierda a derecha o de derecha a izquierda) para tareas como la traducción.

#### Enfoque de Preentrenamiento
El preentrenamiento de BERT involucra modelado del lenguaje enmascarado y predicción de la siguiente oración, lo cual no es una característica del Transformador original.

#### Ajuste Fino para Tareas Específicas
El Transformador original no enfatiza el paso de ajuste fino tanto como BERT, que está diseñado para adaptarse a varias tareas de PNL con ajustes mínimos específicos de la tarea.


In [ ]:
!pip install torch transformers pandas numpy nltk


In [ ]:

import pandas as pd
import numpy as np

import torch

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_csv('/content/drive/MyDrive/news.csv')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
torch.cuda.empty_cache()

In [ ]:

#Limpieza basica del texto, remover puntuación y digitos como fechas, números de usuario de twitter etc.

stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer('[\'a-zA-Z]+') # Acá es para eligir solo parabras del alfabeto entre A-Z minuscula o mayscula, es una RE
lemmatizer = WordNetLemmatizer() #Acá iremos reducir las palabras para su clasificación minima, la raíz semantica de la palabra.


primera_noticia = df.iloc[0]
def preprocess_text(text):
    words = []
    for sentence in sent_tokenize(text):
        tokens = [word for word in tokenizer.tokenize(sentence)]
        tokens = [token.lower() for token in tokens]
        tokens = [token for token in tokens if token not in stop_words]
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        words += tokens
    return ' '.join(words)

df['preprocessado'] = df['title'] + " " + df['text']
df['preprocessado'] = df['preprocessado'].apply(preprocess_text)

# Tokenizador de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preparar datos para BERT
def prepare_data_for_bert(texts, tokenizer, max_length):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_data = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded_data['input_ids'])
        attention_masks.append(encoded_data['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

max_length = 256  # Ajusta esto según la longitud de tus textos
input_ids, attention_masks = prepare_data_for_bert(df['preprocessado'], tokenizer, max_length)

# Etiquetas
labels = torch.tensor(df['authenticity'].apply(lambda x: 0 if x == 'Fake' else 1).values)

# Dataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# Dividir el dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
# Cargar modelo BERT para clasificación
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2, # Clasificación binaria
    output_attentions = False,
    output_hidden_states = False,
)

# Mover modelo al dispositivo GPU
model.cuda()

# Configurar el optimizador
optimizer = AdamW(model.parameters(), lr=2e-5)
# https://arxiv.org/pdf/1711.05101.pdf
# Total de pasos de entrenamiento es número de batches * número de épocas
total_steps = len(train_loader) * 4

# Creador del schedule para el learning rate
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Número de épocas
epochs = 2
acumulation_steps = 2
# Bucle de entrenamiento
for epoch_i in range(0, epochs):
    # Entrenar
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_loader):


        # Desempaquetar los datos del dataloader
        b_input_ids = batch[0].cuda()
        b_input_mask = batch[1].cuda()
        b_labels = batch[2].cuda()

        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()

        # Backward pass
        loss.backward()

        # Prevenir el problema del exploding gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Actualización de parámetros y step del scheduler
        if (step + 1) % acumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            model.zero_grad()
    # Cálculo de la pérdida promedio sobre la época
    avg_train_loss = total_loss / len(train_loader)
    print(f"Promedio de pérdida en entrenamiento: {avg_train_loss}")

    # Evaluación
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in val_loader:
        batch = tuple(t.cuda() for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print(f"Exactitud en validación: {eval_accuracy/nb_eval_steps}")


Promedio de pérdida en entrenamiento: 0.1422567501880694
Exactitud en validación: 0.9965277777777778
Promedio de pérdida en entrenamiento: 0.0023303805838222618
Exactitud en validación: 0.9985119047619048
